In [ ]:
"Получается очень весело\nПочитать почитал, делать делал, а пройти через считывание не могу\nИтого все что после считывания мб не работает))"

In [ ]:
"Github(1 commit ибо уже было поздно перестирать 10 раз): https://github.com/Mr-Maxraw/ML-lab3.git"

In [36]:
import numpy as np #classic
import pandas as pd #classic
import json # преобразование input
import seaborn as sns
from pandas.io.json import json_normalize # нормализация input
from sklearn import preprocessing # /работы с категориями

In [37]:
def cutTo10(obj): # оптимизация по скорости(-10% строк)
    preCut = np.random.rand(len(obj)) < 0.1
    res = obj[preCut]
    return res

In [38]:
def jsonToDf(df, JSON_COLUMNS): # json -> df
    for column in JSON_COLUMNS:
        columnAsDf = json_normalize(df[column])
        columnAsDf.columns = [f'{column}.{subcolumn}' for subcolumn in columnAsDf.columns]
        df = df.drop(column, axis=1).merge(columnAsDf, right_index=True, left_index=True)
    return df

In [39]:
def prepData(df):        
    # df['date'] = pd.to_datetime(df['date'], format='%Y%m%d') # get dates
    
    
    df = df.replace({'socialEngagementType': {'Engaged': True, 'Not Socially Engaged': False}})  # convert to boolean 
    # df = df.replace({'device.isMobile': {True: 1, False: 0}})
    
    floatCol = ['totals.pageviews','totals.bounces','totals.transactionRevenue'] # float converts

    for col in floatCol:
        df[col].fillna(0, inplace=True)
        df[col] = df[col].astype(float)
    df['totals.transactionRevenue'] = np.log1p(df['totals.transactionRevenue'].values)  # расскажите почему логарифм
    
    stringCols = ['device.browser','geoNetwork.subContinent','geoNetwork.region',
                'geoNetwork.networkDomain','trafficSource.medium'] # str conversion

    for col in stringCols:
        df = df.replace({col: {'not available in demo dataset': '(ng)'}}) #"Не доступно в демосете" -> ng(not given)
        le = preprocessing.LabelEncoder()
        le.fit(df[col].unique())
        df[col] = le.transform(list(df[col].values.astype('str')))
        df[col] = df[col].astype(float)
    

    for columns in df.columns: # убираем объекты и числа
        if df[columns].dtype == 'object' or df[columns].dtype == 'int64':
            df = df.drop([columns], axis = 1)
    
    return df

In [40]:
#Импорт и чистка
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
data = pd.read_csv("C:\HSE\Машинка\Лаба3\\train.csv", sep = ",",  dtype = {'fullVisitorId': 'str'},
                   converters={column: json.loads for column in JSON_COLUMNS}, 
                   nrows = None)
data = jsonToDf(data, JSON_COLUMNS) # преобразование input
data = cutTo10(data) # оптимизация по скорости

delete = data.ne('not available in demo dataset')
data = data.loc[delete.any(1), delete.any()]

MemoryError: Unable to allocate 512. KiB for an array with shape (65536,) and data type float64

In [41]:
data = prepData(data)

NameError: name 'data' is not defined

In [42]:
# Сам МЛ
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [43]:
#Модельки и разбивка
trg = data['totals.transactionRevenue']  # Отделяем зависимую переменную 
trn = data.drop(columns=['totals.transactionRevenue'], axis=1)
Xtrn, Xtest, Ytrn, Ytest = train_test_split(trn, trg, test_size = 0.2) # Разбиваем выборку

models = [GradientBoostingRegressor(), # Бустинг
          RandomForestRegressor(), # Случайный лес
          LinearRegression() # Линейная регрессия
	     ]

NameError: name 'data' is not defined

In [44]:
# создаем коллекции
res = pd.DataFrame() 
tmp = pd.DataFrame() 

In [45]:
for model in models:
   name = str(model)
   modelName = name[:name.index('(')]  
   # Кросс валидация
   scores = np.array([])
   for i in range(10):  # Тут теперь 10 разбиений
       fold = KFold(n_splits=5, shuffle=True)
       scoresOnSplit = cross_val_score(
               estimator = model, X=Xtest, y=Ytest,
               cv=fold, scoring='r2')
       scores = np.append(scores, scoresOnSplit)
   print(modelName, 'is done') 
   tmp['Оценки качества'] = pd.Series(scores)
   tmp['Модель'] = modelName
   res = res.append([tmp])

NameError: name 'models' is not defined

In [46]:
#FINALLY
res = res.replace({'Модель': {'GradientBoostingRegressor': 'Бустинг', 'RandomForestRegressor': 'Лес', 'LinearRegression': 'Лин. Регрессия'}})
sns.boxplot(y = 'Оценки качества', x = 'Модель', data = res)   

ValueError: Could not interpret input 'Модель'